In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy import ndimage
import xml
import xml.etree.ElementTree as ET
import cv2
import warnings
from skimage.transform import resize

In [ ]:
# I think we need the sum of the density from the density maps for each corresponding subwindow?

In [ ]:
def chose_xml_and_jpeg(file_location):
    # list all files in location
    list_of_all_files = os.listdir(file_location)
    # sort files
    list_of_all_files.sort()
    # separate xml and jpeg files
    all_xml_files = [file for file in list_of_all_files if file.split('.')[-1] == 'xml']
    all_xml_files.sort()
    all_jpeg_files = [file for file in list_of_all_files if file not in all_xml_files]
    all_jpeg_files.sort()
    # get the final 20 files
    chosen_xml_files = all_xml_files[-20:]
    chosen_jpeg_files = all_jpeg_files[-20:]
    # make sure the xml and jpeg files correspond to each other?
    mean = np.mean([file.split('.')[0] for file in chosen_xml_files] == [file.split('.')[0] for file in chosen_jpeg_files])
    return(chosen_jpeg_files, chosen_xml_files, mean)

In [ ]:
# for horizontally annotated images
def get_density_maps_horizontal(file_name, image_path, xml_path):
    xml_file = file_name + '.xml'
    xml_file_path = os.path.join(xml_path, xml_file)

    # Get coords from the xml file
    # parse the xml file
    parsed_file = ET.parse(xml_file_path)
    # get the roots
    root = parsed_file.getroot()
    # get the roots here
    coords = []
    for child in root:
        for i in child:
            for j in i:
                coords.append(int(j.text))
    
    # chunk the points into sets of 4 - these are the coordinates of the bounding boxes
    points_tupples = []
    for i in range(0, len(coords), 4):
        points_tupples.append(coords[i:i + 4])

    # make a dataframe with these points
    coords_df = pd.DataFrame(points_tupples, columns = ["bleft_x", "bleft_y", "tright_x", "tright_y"])

    # compute the number of tassels in each image
    no_of_tassels = len(points_tupples)

    # compute the mid coordinates
    coords_df["mid_x"] = (round(0.5*(coords_df["bleft_x"] + coords_df["tright_x"]))).astype(int)
    coords_df["mid_y"] = (round(0.5*(coords_df["bleft_y"] + coords_df["tright_y"]))).astype(int)

    # extract the mid cordinates
    mid_coords = coords_df[["mid_x", "mid_y"]]
    # cap the coords at the max height and width values
    mid_coords.loc[mid_coords['mid_x'] > 1024, 'mid_x'] = 1023
    mid_coords.loc[mid_coords['mid_y'] > 768, 'mid_y'] = 767
    warnings.filterwarnings("ignore")

    # plot the bounding boxes on images
    # get image name and path
    image_name = file_name + '.jpeg'
    imge_file_path = os.path.join(image_path, image_name)
    # read the image
    read_image = plt.imread(imge_file_path)
    read_image = ndimage.rotate(read_image, 0)

    # check the shape of the read image
    read_image_shape = read_image.shape
    #  plot the bounding boxes on the image
    for points in points_tupples:
        annotated_image = cv2.rectangle(read_image, (points[0],points[1]), (points[2],points[3]), color = (255,0,0), thickness = 2)
    # plt.figure(figsize = (12,18))
    plt.imshow(annotated_image)
    plt.show()

    # plot the mid points on the image
    coords_list = mid_coords.values.tolist()
    # read the image again
    read_image_again = plt.imread(imge_file_path)
    read_image_again = ndimage.rotate(read_image_again, 0)
    # draw the circles on image
    for i in coords_list:
        image_with_mids = cv2.circle(read_image_again, i, radius=5, color=(255, 0, 0), thickness=-1)
    # look at the annotated image
    # plt.figure(figsize = (12,18))
    plt.imshow(image_with_mids)
    plt.show()

    # also try creating the density map here
    # first create the empty maps
    np_image = np.zeros((read_image_shape[0], read_image_shape[1]))
    # get the dot maps
    for point in coords_list:
        np_image[point[1], point[0]] = 1
    # plot the image
    # plt.figure(figsize = (12,18))
    plt.imshow(np_image, cmap = "Greys")
    plt.show()

    # now define the kernel and run the convolution
    one_d_kerenel = cv2.getGaussianKernel(50,5)
    two_d_kernel = np.multiply(one_d_kerenel.T, one_d_kerenel)

    # Shape of the 2D kernel
    twoD_shape = two_d_kernel.shape
        
    # do the convolution
    convolution = ndimage.convolve(np_image, two_d_kernel)
        
    # plot the density map
    # plt.figure(figsize = (12,18))
    plt.imshow(convolution, cmap = "Greys")
    plt.show()
        
    # get the sums of the images
    img_sum = np.sum(convolution)

    return(file_name, read_image_shape, no_of_tassels, img_sum, convolution)

In [ ]:
def create_densities_for_subwindows(density_map, stride = 30, kernel_size = 30):
        
    # create the counts for the subwindows as follows
    img_height = density_map.shape[0]
    img_width = density_map.shape[1]
    
    density_sums = []
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            # sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            density = density_map[i: i + kernel_size, j : j + kernel_size]
            dense_sum = np.sum(density)
            density_sums.append(dense_sum)

    print("sum of the convolved map: ", np.sum(density_sums))
    density_sums_array = np.array(density_sums).reshape(-1,1)
    return density_sums_array

In [ ]:
# Block 0103

In [ ]:
block_0103 = '../../../Spring_2024/S_lab_TasselNet/Block_3_TN/Block_3_images_and_xml'

In [ ]:
# get im and xml files
block_0103_target_ims, block_0103_target_xmls, block_0103_mean = chose_xml_and_jpeg(block_0103)

In [ ]:
len(block_0103_target_ims)

In [ ]:
# block_0103_target_xmls

In [ ]:
block_0103_mean

In [ ]:
# just file names
block_0103_file_names = [file.split(".")[0] for file in block_0103_target_ims]

In [ ]:
# block_0103_file_names

In [ ]:
%%time
# Get the density maps
block_0103_names = []
block_0103_im_shapes = []
block_0103_tassel_count = []
block_0103_tassel_count_alt = []
# capture the density maps here
block_0103_convs = []
for file in block_0103_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0103, block_0103)
    block_0103_names.append(name)
    block_0103_im_shapes.append(shape)
    block_0103_tassel_count.append(no_tassels)
    block_0103_tassel_count_alt.append(no_tassel_alt)
    block_0103_convs.append(conv)

In [ ]:
# block_0103_names

In [ ]:
# block_0103_im_shapes

In [ ]:
# block_0103_tassel_count, block_0103_tassel_count_alt

In [ ]:
np.mean(np.round(block_0103_tassel_count, 0) == np.round(block_0103_tassel_count_alt, 0))

In [ ]:
%%time
# Get the stacks?
block_0103_density_list = []
for conv in block_0103_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0103_density_list.append(density_values)

In [ ]:
# sum of the conv maps arent gonna make any sense here as we have overlapping sub-windows

In [ ]:
len(block_0103_density_list)

In [ ]:
block_0103_density_list[0].shape

In [ ]:
# need to stack this list
block_0103_all_density_stack = np.stack(block_0103_density_list, axis = 1)

In [ ]:
block_0103_all_density_stack.shape

In [ ]:
np.array(block_0103_tassel_count_alt)

In [ ]:
np.sum(block_0103_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
block_0103_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0103_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0103_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0103_df = pd.DataFrame(zip(time_period_name, block_0103_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0103_df.head()

In [ ]:
true_densities_0103_df.shape

In [ ]:
true_densities_0103_df.to_csv("data/test_block_true_counts/block_0103_true_counts.csv", index = False)

In [ ]:
block_0103_all_density_stack.shape

In [ ]:
len(block_0103_density_list)

In [ ]:
%%time
for i in range(len(block_0103_density_list)):
    print(np.mean(block_0103_density_list[i] == block_0103_all_density_stack[:, i, :]))   

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0103.npy", block_0103_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0103 = np.load("data/densities_overlapping/all_densities_block_0103.npy")

In [ ]:
np.mean(loaded_density_blk_0103 == block_0103_all_density_stack)

In [ ]:
# Block 0104

In [ ]:
block_0104 = '../../../Spring_2024/S_lab_TasselNet/Block_4_TN/Block_4_images_and_xml'

In [ ]:
# get im and xml files
block_0104_target_ims, block_0104_target_xmls, block_0104_mean = chose_xml_and_jpeg(block_0104)

In [ ]:
# block_0104_target_ims

In [ ]:
# block_0104_target_xmls

In [ ]:
block_0104_mean

In [ ]:
# just file names
block_0104_file_names = [file.split(".")[0] for file in block_0104_target_ims]

In [ ]:
# block_0104_file_names

In [ ]:
%%time
# Get the density maps
block_0104_names = []
block_0104_im_shapes = []
block_0104_tassel_count = []
block_0104_tassel_count_alt = []
# capture the density maps here
block_0104_convs = []
for file in block_0104_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0104, block_0104)
    block_0104_names.append(name)
    block_0104_im_shapes.append(shape)
    block_0104_tassel_count.append(no_tassels)
    block_0104_tassel_count_alt.append(no_tassel_alt)
    block_0104_convs.append(conv)

In [ ]:
block_0104_names

In [ ]:
block_0104_im_shapes

In [ ]:
np.mean(np.round(block_0104_tassel_count, 0) == np.round(block_0104_tassel_count_alt, 0))

In [ ]:
%%time
# Get the stacks?
block_0104_density_list = []
for conv in block_0104_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0104_density_list.append(density_values)

In [ ]:
len(block_0104_density_list)

In [ ]:
block_0104_density_list[0].shape

In [ ]:
# need to stack this list
block_0104_all_density_stack = np.stack(block_0104_density_list, axis = 1)

In [ ]:
block_0104_all_density_stack.shape

In [ ]:
np.array(block_0104_tassel_count_alt)

In [ ]:
np.sum(block_0104_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0104_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0104_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0104_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0104_df = pd.DataFrame(zip(time_period_name, block_0104_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0104_df.head()

In [ ]:
true_densities_0104_df.to_csv("data/test_block_true_counts/block_0104_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0104_all_density_stack.shape

In [ ]:
len(block_0104_density_list)

In [ ]:
%%time
for i in range(len(block_0104_density_list)):
    print(np.mean(block_0104_density_list[i] == block_0104_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly done

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0104_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0104.npy", block_0104_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0104 = np.load("data/densities_overlapping/all_densities_block_0104.npy")

In [ ]:
np.mean(loaded_density_blk_0104 == block_0104_all_density_stack)

In [ ]:
# Block 0105

In [ ]:
block_0105 = '../../../Spring_2024/S_lab_TasselNet/Block_5_TN/Block_5_images_and_xml'

In [ ]:
# get im and xml files
block_0105_target_ims, block_0105_target_xmls, block_0105_mean = chose_xml_and_jpeg(block_0105)

In [ ]:
block_0105_target_ims

In [ ]:
block_0105_target_xmls

In [ ]:
block_0105_mean

In [ ]:
# just file names
block_0105_file_names = [file.split(".")[0] for file in block_0105_target_ims]

In [ ]:
block_0105_file_names

In [ ]:
%%time
# Get the density maps
block_0105_names = []
block_0105_im_shapes = []
block_0105_tassel_count = []
block_0105_tassel_count_alt = []
# capture the density maps here
block_0105_convs = []
for file in block_0105_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0105, block_0105)
    block_0105_names.append(name)
    block_0105_im_shapes.append(shape)
    block_0105_tassel_count.append(no_tassels)
    block_0105_tassel_count_alt.append(no_tassel_alt)
    block_0105_convs.append(conv)

In [ ]:
block_0105_names

In [ ]:
block_0105_im_shapes

In [ ]:
block_0105_tassel_count, block_0105_tassel_count_alt

In [ ]:
np.mean(np.round(block_0105_tassel_count, 0) == np.round(block_0105_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0105_density_list = []
for conv in block_0105_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0105_density_list.append(density_values)

In [ ]:
len(block_0105_density_list)

In [ ]:
block_0105_density_list[0].shape

In [ ]:
# need to stack this list
block_0105_all_density_stack = np.stack(block_0105_density_list, axis = 1)

In [ ]:
block_0105_all_density_stack.shape

In [ ]:
np.array(block_0105_tassel_count_alt)

In [ ]:
np.sum(block_0105_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0105_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0105_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0105_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0105_df = pd.DataFrame(zip(time_period_name, block_0105_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0105_df.head()

In [ ]:
true_densities_0105_df.to_csv("data/test_block_true_counts/block_0105_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0105_all_density_stack.shape

In [ ]:
len(block_0105_density_list)

In [ ]:
%%time
for i in range(len(block_0105_density_list)):
    print(np.mean(block_0105_density_list[i] == block_0105_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0105_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0105.npy", block_0105_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0105 = np.load("data/densities_overlapping/all_densities_block_0105.npy")

In [ ]:
np.mean(loaded_density_blk_0105 == block_0105_all_density_stack)

In [ ]:
# Block 0106

In [ ]:
block_0106 = '../../../Spring_2024/S_lab_TasselNet/Block_6_TN/Block_6_images_and_xml'

In [ ]:
# get im and xml files
block_0106_target_ims, block_0106_target_xmls, block_0106_mean = chose_xml_and_jpeg(block_0106)

In [ ]:
block_0106_target_ims

In [ ]:
block_0106_target_xmls

In [ ]:
block_0106_mean

In [ ]:
# just file names
block_0106_file_names = [file.split(".")[0] for file in block_0106_target_ims]

In [ ]:
block_0106_file_names

In [ ]:
%%time
# Get the density maps
block_0106_names = []
block_0106_im_shapes = []
block_0106_tassel_count = []
block_0106_tassel_count_alt = []
# capture the density maps here
block_0106_convs = []
for file in block_0106_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0106, block_0106)
    block_0106_names.append(name)
    block_0106_im_shapes.append(shape)
    block_0106_tassel_count.append(no_tassels)
    block_0106_tassel_count_alt.append(no_tassel_alt)
    block_0106_convs.append(conv)

In [ ]:
block_0106_names

In [ ]:
block_0106_im_shapes

In [ ]:
# block_0106_tassel_count, block_0106_tassel_count_alt

In [ ]:
np.mean(np.round(block_0106_tassel_count, 0) == np.round(block_0106_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0106_density_list = []
for conv in block_0106_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0106_density_list.append(density_values)

In [ ]:
len(block_0106_density_list)

In [ ]:
block_0106_density_list[0].shape

In [ ]:
# need to stack this list
block_0106_all_density_stack = np.stack(block_0106_density_list, axis = 1)

In [ ]:
block_0106_all_density_stack.shape

In [ ]:
np.array(block_0106_tassel_count_alt)

In [ ]:
np.sum(block_0106_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0106_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0106_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0106_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0106_df = pd.DataFrame(zip(time_period_name, block_0106_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0106_df.head()

In [ ]:
true_densities_0106_df.to_csv("data/test_block_true_counts/block_0106_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0106_all_density_stack.shape

In [ ]:
len(block_0106_density_list)

In [ ]:
%%time
for i in range(len(block_0106_density_list)):
    print(np.mean(block_0106_density_list[i] == block_0106_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0106_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0106.npy", block_0106_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0106 = np.load("data/densities_overlapping/all_densities_block_0106.npy")

In [ ]:
np.mean(loaded_density_blk_0106 == block_0106_all_density_stack)

In [ ]:
# Block 0201

In [ ]:
block_0201 = '../../../Spring_2024/S_lab_TasselNet/Block_7_TN/Block_7_images_and_xml'

In [ ]:
# get im and xml files
block_0201_target_ims, block_0201_target_xmls, block_0201_mean = chose_xml_and_jpeg(block_0201)

In [ ]:
# block_0201_target_ims

In [ ]:
# block_0201_target_xmls

In [ ]:
block_0201_mean

In [ ]:
# just file names
block_0201_file_names = [file.split(".")[0] for file in block_0201_target_ims]

In [ ]:
# block_0201_file_names

In [ ]:
%%time
# Get the density maps
block_0201_names = []
block_0201_im_shapes = []
block_0201_tassel_count = []
block_0201_tassel_count_alt = []
# capture the density maps here
block_0201_convs = []
for file in block_0201_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0201, block_0201)
    block_0201_names.append(name)
    block_0201_im_shapes.append(shape)
    block_0201_tassel_count.append(no_tassels)
    block_0201_tassel_count_alt.append(no_tassel_alt)
    block_0201_convs.append(conv)

In [ ]:
block_0201_names

In [ ]:
block_0201_im_shapes

In [ ]:
block_0201_tassel_count, block_0201_tassel_count_alt

In [ ]:
np.mean(np.round(block_0201_tassel_count, 0) == np.round(block_0201_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0201_density_list = []
for conv in block_0201_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0201_density_list.append(density_values)

In [ ]:
len(block_0201_density_list)

In [ ]:
block_0201_density_list[0].shape

In [ ]:
# need to stack this list
block_0201_all_density_stack = np.stack(block_0201_density_list, axis = 1)

In [ ]:
block_0201_all_density_stack.shape

In [ ]:
np.array(block_0201_tassel_count_alt)

In [ ]:
np.sum(block_0201_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0201_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0201_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0201_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0201_df = pd.DataFrame(zip(time_period_name, block_0201_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0201_df.head()

In [ ]:
true_densities_0201_df.to_csv("data/test_block_true_counts/block_0201_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0201_all_density_stack.shape

In [ ]:
len(block_0201_density_list)

In [ ]:
%%time
for i in range(len(block_0201_density_list)):
    print(np.mean(block_0201_density_list[i] == block_0201_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0201_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0201.npy", block_0201_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0201 = np.load("data/densities_overlapping/all_densities_block_0201.npy")

In [ ]:
np.mean(loaded_density_blk_0201 == block_0201_all_density_stack)

In [ ]:
# Block 0202

In [ ]:
block_0202 = '../../../Spring_2024/S_lab_TasselNet/Block_8_TN/Block_8_images_and_xml'

In [ ]:
# get im and xml files
block_0202_target_ims, block_0202_target_xmls, block_0202_mean = chose_xml_and_jpeg(block_0202)

In [ ]:
block_0202_target_ims

In [ ]:
block_0202_target_xmls

In [ ]:
block_0202_mean

In [ ]:
# just file names
block_0202_file_names = [file.split(".")[0] for file in block_0202_target_ims]

In [ ]:
block_0202_file_names

In [ ]:
%%time
# Get the density maps
block_0202_names = []
block_0202_im_shapes = []
block_0202_tassel_count = []
block_0202_tassel_count_alt = []
# capture the density maps here
block_0202_convs = []
for file in block_0202_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0202, block_0202)
    block_0202_names.append(name)
    block_0202_im_shapes.append(shape)
    block_0202_tassel_count.append(no_tassels)
    block_0202_tassel_count_alt.append(no_tassel_alt)
    block_0202_convs.append(conv)

In [ ]:
block_0202_names

In [ ]:
block_0202_im_shapes

In [ ]:
block_0202_tassel_count, block_0202_tassel_count_alt

In [ ]:
np.mean(np.round(block_0202_tassel_count, 0) == np.round(block_0202_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0202_density_list = []
for conv in block_0202_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0202_density_list.append(density_values)

In [ ]:
len(block_0202_density_list)

In [ ]:
block_0202_density_list[0].shape

In [ ]:
# need to stack this list
block_0202_all_density_stack = np.stack(block_0202_density_list, axis = 1)

In [ ]:
block_0202_all_density_stack.shape

In [ ]:
np.array(block_0202_tassel_count_alt)

In [ ]:
np.sum(block_0202_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0202_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0202_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0202_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0202_df = pd.DataFrame(zip(time_period_name, block_0202_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0202_df.head()

In [ ]:
true_densities_0202_df.to_csv("data/test_block_true_counts/block_0202_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0202_all_density_stack.shape

In [ ]:
len(block_0202_density_list)

In [ ]:
%%time
for i in range(len(block_0202_density_list)):
    print(np.mean(block_0202_density_list[i] == block_0202_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0202_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0202.npy", block_0202_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0202 = np.load("data/densities_overlapping/all_densities_block_0202.npy")

In [ ]:
np.mean(loaded_density_blk_0202 == block_0202_all_density_stack)

In [ ]:
# Block 0205

In [ ]:
block_0205 = '../../../Spring_2024/S_lab_TasselNet/Block_11_TN/Block_11_images_and_xml'

In [ ]:
# get im and xml files
block_0205_target_ims, block_0205_target_xmls, block_0205_mean = chose_xml_and_jpeg(block_0205)

In [ ]:
block_0205_target_ims

In [ ]:
block_0205_target_xmls

In [ ]:
block_0205_mean

In [ ]:
# just file names
block_0205_file_names = [file.split(".")[0] for file in block_0205_target_ims]

In [ ]:
block_0205_file_names

In [ ]:
%%time
# Get the density maps
block_0205_names = []
block_0205_im_shapes = []
block_0205_tassel_count = []
block_0205_tassel_count_alt = []
# capture the density maps here
block_0205_convs = []
for file in block_0205_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0205, block_0205)
    block_0205_names.append(name)
    block_0205_im_shapes.append(shape)
    block_0205_tassel_count.append(no_tassels)
    block_0205_tassel_count_alt.append(no_tassel_alt)
    block_0205_convs.append(conv)

In [ ]:
block_0205_names

In [ ]:
block_0205_im_shapes

In [ ]:
block_0205_tassel_count, block_0205_tassel_count_alt

In [ ]:
np.mean(np.round(block_0205_tassel_count, 0) == np.round(block_0205_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0205_density_list = []
for conv in block_0205_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0205_density_list.append(density_values)

In [ ]:
len(block_0205_density_list)

In [ ]:
block_0205_density_list[0].shape

In [ ]:
# need to stack this list
block_0205_all_density_stack = np.stack(block_0205_density_list, axis = 1)

In [ ]:
block_0205_all_density_stack.shape

In [ ]:
np.array(block_0205_tassel_count_alt)

In [ ]:
np.sum(block_0205_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0205_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0205_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0205_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0205_df = pd.DataFrame(zip(time_period_name, block_0205_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0205_df.head()

In [ ]:
true_densities_0205_df.to_csv("data/test_block_true_counts/block_0205_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0205_all_density_stack.shape

In [ ]:
len(block_0205_density_list)

In [ ]:
%%time
for i in range(len(block_0205_density_list)):
    print(np.mean(block_0205_density_list[i] == block_0205_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly done

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0205_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0205.npy", block_0205_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0205 = np.load("data/densities_overlapping/all_densities_block_0205.npy")

In [ ]:
np.mean(loaded_density_blk_0205 == block_0205_all_density_stack)

In [ ]:
# Block 0206

In [ ]:
block_0206 = '../../../Spring_2024/S_lab_TasselNet/Block_12_TN/Block_12_images_and_xml'

In [ ]:
# get im and xml files
block_0206_target_ims, block_0206_target_xmls, block_0206_mean = chose_xml_and_jpeg(block_0206)

In [ ]:
block_0206_target_ims

In [ ]:
block_0206_target_xmls

In [ ]:
block_0206_mean

In [ ]:
# just file names
block_0206_file_names = [file.split(".")[0] for file in block_0206_target_ims]

In [ ]:
block_0206_file_names

In [ ]:
%%time
# Get the density maps
block_0206_names = []
block_0206_im_shapes = []
block_0206_tassel_count = []
block_0206_tassel_count_alt = []
# capture the density maps here
block_0206_convs = []
for file in block_0206_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0206, block_0206)
    block_0206_names.append(name)
    block_0206_im_shapes.append(shape)
    block_0206_tassel_count.append(no_tassels)
    block_0206_tassel_count_alt.append(no_tassel_alt)
    block_0206_convs.append(conv)

In [ ]:
block_0206_names

In [ ]:
block_0206_im_shapes

In [ ]:
block_0206_tassel_count, block_0206_tassel_count_alt

In [ ]:
np.mean(np.round(block_0206_tassel_count, 0) == np.round(block_0206_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0206_density_list = []
for conv in block_0206_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0206_density_list.append(density_values)

In [ ]:
len(block_0206_density_list)

In [ ]:
block_0206_density_list[0].shape

In [ ]:
# need to stack this list
block_0206_all_density_stack = np.stack(block_0206_density_list, axis = 1)

In [ ]:
block_0206_all_density_stack.shape

In [ ]:
np.array(block_0206_tassel_count_alt)

In [ ]:
np.sum(block_0206_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0206_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0206_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0206_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0206_df = pd.DataFrame(zip(time_period_name, block_0206_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0206_df.head()

In [ ]:
true_densities_0206_df.to_csv("data/test_block_true_counts/block_0206_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0206_all_density_stack.shape

In [ ]:
len(block_0206_density_list)

In [ ]:
%%time
for i in range(len(block_0206_density_list)):
    print(np.mean(block_0206_density_list[i] == block_0206_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0206_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0206.npy", block_0206_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0206 = np.load("data/densities_overlapping/all_densities_block_0206.npy")

In [ ]:
np.mean(loaded_density_blk_0206 == block_0206_all_density_stack)

In [ ]:
# Block 0302

In [ ]:
block_0302 = '../../../Spring_2024/S_lab_TasselNet/Block_14_TN/Block_14_images_and_xml'

In [ ]:
# get im and xml files
block_0302_target_ims, block_0302_target_xmls, block_0302_mean = chose_xml_and_jpeg(block_0302)

In [ ]:
block_0302_target_ims

In [ ]:
block_0302_target_xmls

In [ ]:
block_0302_mean

In [ ]:
# just file names
block_0302_file_names = [file.split(".")[0] for file in block_0302_target_ims]

In [ ]:
block_0302_file_names

In [ ]:
%%time
# Get the density maps
block_0302_names = []
block_0302_im_shapes = []
block_0302_tassel_count = []
block_0302_tassel_count_alt = []
# capture the density maps here
block_0302_convs = []
for file in block_0302_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0302, block_0302)
    block_0302_names.append(name)
    block_0302_im_shapes.append(shape)
    block_0302_tassel_count.append(no_tassels)
    block_0302_tassel_count_alt.append(no_tassel_alt)
    block_0302_convs.append(conv)

In [ ]:
block_0302_names

In [ ]:
block_0302_im_shapes

In [ ]:
block_0302_tassel_count, block_0302_tassel_count_alt

In [ ]:
np.mean(np.round(block_0302_tassel_count, 0) == np.round(block_0302_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0302_density_list = []
for conv in block_0302_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0302_density_list.append(density_values)

In [ ]:
len(block_0302_density_list)

In [ ]:
block_0302_density_list[0].shape

In [ ]:
# need to stack this list
block_0302_all_density_stack = np.stack(block_0302_density_list, axis = 1)

In [ ]:
block_0302_all_density_stack.shape

In [ ]:
np.array(block_0302_tassel_count_alt)

In [ ]:
np.sum(block_0302_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0302_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0302_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0302_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0302_df = pd.DataFrame(zip(time_period_name, block_0302_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0302_df.head()

In [ ]:
true_densities_0302_df.to_csv("data/test_block_true_counts/block_0302_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0302_all_density_stack.shape

In [ ]:
len(block_0302_density_list)

In [ ]:
%%time
for i in range(len(block_0302_density_list)):
    print(np.mean(block_0302_density_list[i] == block_0302_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0302_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0302.npy", block_0302_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0302 = np.load("data/densities_overlapping/all_densities_block_0302.npy")

In [ ]:
np.mean(loaded_density_blk_0302 == block_0302_all_density_stack)

In [ ]:
# Block 0303

In [ ]:
block_0303 = '../../../Spring_2024/S_lab_TasselNet/Block_15_TN/Block_15_images_and_xml'

In [ ]:
# get im and xml files
block_0303_target_ims, block_0303_target_xmls, block_0303_mean = chose_xml_and_jpeg(block_0303)

In [ ]:
block_0303_target_ims

In [ ]:
block_0303_target_xmls

In [ ]:
block_0303_mean

In [ ]:
# just file names
block_0303_file_names = [file.split(".")[0] for file in block_0303_target_ims]

In [ ]:
block_0303_file_names

In [ ]:
%%time
# Get the density maps
block_0303_names = []
block_0303_im_shapes = []
block_0303_tassel_count = []
block_0303_tassel_count_alt = []
# capture the density maps here
block_0303_convs = []
for file in block_0303_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0303, block_0303)
    block_0303_names.append(name)
    block_0303_im_shapes.append(shape)
    block_0303_tassel_count.append(no_tassels)
    block_0303_tassel_count_alt.append(no_tassel_alt)
    block_0303_convs.append(conv)

In [ ]:
block_0303_names

In [ ]:
block_0303_im_shapes

In [ ]:
block_0303_tassel_count, block_0303_tassel_count_alt

In [ ]:
np.mean(np.round(block_0303_tassel_count, 0) == np.round(block_0303_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0303_density_list = []
for conv in block_0303_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0303_density_list.append(density_values)

In [ ]:
len(block_0303_density_list)

In [ ]:
block_0303_density_list[0].shape

In [ ]:
# need to stack this list
block_0303_all_density_stack = np.stack(block_0303_density_list, axis = 1)

In [ ]:
block_0303_all_density_stack.shape

In [ ]:
np.array(block_0303_tassel_count_alt)

In [ ]:
np.sum(block_0303_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0303_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0303_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0303_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0303_df = pd.DataFrame(zip(time_period_name, block_0303_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0303_df.head()

In [ ]:
true_densities_0303_df.to_csv("data/test_block_true_counts/block_0303_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0303_all_density_stack.shape

In [ ]:
len(block_0303_density_list)

In [ ]:
%%time
for i in range(len(block_0303_density_list)):
    print(np.mean(block_0303_density_list[i] == block_0303_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0303_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0303.npy", block_0303_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0303 = np.load("data/densities_overlapping/all_densities_block_0303.npy")

In [ ]:
np.mean(loaded_density_blk_0303 == block_0303_all_density_stack)

In [ ]:
# Block 0304

In [ ]:
block_0304 = '../../../Spring_2024/S_lab_TasselNet/Block_16_TN/Block_16_images_and_xml'

In [ ]:
# get im and xml files
block_0304_target_ims, block_0304_target_xmls, block_0304_mean = chose_xml_and_jpeg(block_0304)

In [ ]:
block_0304_target_ims

In [ ]:
block_0304_target_xmls

In [ ]:
block_0304_mean

In [ ]:
# just file names
block_0304_file_names = [file.split(".")[0] for file in block_0304_target_ims]

In [ ]:
block_0304_file_names

In [ ]:
%%time
# Get the density maps
block_0304_names = []
block_0304_im_shapes = []
block_0304_tassel_count = []
block_0304_tassel_count_alt = []
# capture the density maps here
block_0304_convs = []
for file in block_0304_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0304, block_0304)
    block_0304_names.append(name)
    block_0304_im_shapes.append(shape)
    block_0304_tassel_count.append(no_tassels)
    block_0304_tassel_count_alt.append(no_tassel_alt)
    block_0304_convs.append(conv)

In [ ]:
block_0304_names

In [ ]:
block_0304_im_shapes

In [ ]:
block_0304_tassel_count, block_0304_tassel_count_alt

In [ ]:
np.mean(np.round(block_0304_tassel_count, 0) == np.round(block_0304_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0304_density_list = []
for conv in block_0304_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0304_density_list.append(density_values)

In [ ]:
len(block_0304_density_list)

In [ ]:
block_0304_density_list[0].shape

In [ ]:
# need to stack this list
block_0304_all_density_stack = np.stack(block_0304_density_list, axis = 1)

In [ ]:
block_0304_all_density_stack.shape

In [ ]:
np.array(block_0304_tassel_count_alt)

In [ ]:
np.sum(block_0304_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0304_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0304_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0304_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0304_df = pd.DataFrame(zip(time_period_name, block_0304_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0304_df.head()

In [ ]:
true_densities_0304_df.to_csv("data/test_block_true_counts/block_0304_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0304_all_density_stack.shape

In [ ]:
len(block_0304_density_list)

In [ ]:
%%time
for i in range(len(block_0304_density_list)):
    print(np.mean(block_0304_density_list[i] == block_0304_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0304_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0304.npy", block_0304_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0304 = np.load("data/densities_overlapping/all_densities_block_0304.npy")

In [ ]:
np.mean(loaded_density_blk_0304 == block_0304_all_density_stack)

In [ ]:
# Block 0305

In [ ]:
block_0305 = '../../../Spring_2024/S_lab_TasselNet/Block_17_TN/Block_17_images_and_xml'

In [ ]:
# get im and xml files
block_0305_target_ims, block_0305_target_xmls, block_0305_mean = chose_xml_and_jpeg(block_0305)

In [ ]:
block_0305_target_ims

In [ ]:
block_0305_target_xmls

In [ ]:
block_0305_mean

In [ ]:
# just file names
block_0305_file_names = [file.split(".")[0] for file in block_0305_target_ims]

In [ ]:
block_0305_file_names

In [ ]:
%%time
# Get the density maps
block_0305_names = []
block_0305_im_shapes = []
block_0305_tassel_count = []
block_0305_tassel_count_alt = []
# capture the density maps here
block_0305_convs = []
for file in block_0305_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0305, block_0305)
    block_0305_names.append(name)
    block_0305_im_shapes.append(shape)
    block_0305_tassel_count.append(no_tassels)
    block_0305_tassel_count_alt.append(no_tassel_alt)
    block_0305_convs.append(conv)

In [ ]:
block_0305_names

In [ ]:
block_0305_im_shapes

In [ ]:
block_0305_tassel_count, block_0305_tassel_count_alt

In [ ]:
np.mean(np.round(block_0305_tassel_count, 0) == np.round(block_0305_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0305_density_list = []
for conv in block_0305_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0305_density_list.append(density_values)

In [ ]:
len(block_0305_density_list)

In [ ]:
block_0305_density_list[0].shape

In [ ]:
# need to stack this list
block_0305_all_density_stack = np.stack(block_0305_density_list, axis = 1)

In [ ]:
block_0305_all_density_stack.shape

In [ ]:
np.array(block_0305_tassel_count_alt)

In [ ]:
np.sum(block_0305_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0305_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0305_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0305_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0305_df = pd.DataFrame(zip(time_period_name, block_0305_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0305_df.head()

In [ ]:
true_densities_0305_df.to_csv("data/test_block_true_counts/block_0305_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0305_all_density_stack.shape

In [ ]:
len(block_0305_density_list)

In [ ]:
%%time
for i in range(len(block_0305_density_list)):
    print(np.mean(block_0305_density_list[i] == block_0305_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0305_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0305.npy", block_0305_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0305 = np.load("data/densities_overlapping/all_densities_block_0305.npy")

In [ ]:
np.mean(loaded_density_blk_0305 == block_0305_all_density_stack)

In [ ]:
# Block 0306

In [ ]:
block_0306 = '../../../Spring_2024/S_lab_TasselNet/Block_18_TN/Block_18_images_and_xml'

In [ ]:
# get im and xml files
block_0306_target_ims, block_0306_target_xmls, block_0306_mean = chose_xml_and_jpeg(block_0306)

In [ ]:
block_0306_target_ims

In [ ]:
block_0306_target_xmls

In [ ]:
block_0306_mean

In [ ]:
# just file names
block_0306_file_names = [file.split(".")[0] for file in block_0306_target_ims]

In [ ]:
block_0306_file_names

In [ ]:
%%time
# Get the density maps
block_0306_names = []
block_0306_im_shapes = []
block_0306_tassel_count = []
block_0306_tassel_count_alt = []
# capture the density maps here
block_0306_convs = []
for file in block_0306_file_names:
    name, shape, no_tassels, no_tassel_alt, conv = get_density_maps_horizontal(file, block_0306, block_0306)
    block_0306_names.append(name)
    block_0306_im_shapes.append(shape)
    block_0306_tassel_count.append(no_tassels)
    block_0306_tassel_count_alt.append(no_tassel_alt)
    block_0306_convs.append(conv)

In [ ]:
block_0306_names

In [ ]:
block_0306_im_shapes

In [ ]:
block_0306_tassel_count, block_0306_tassel_count_alt

In [ ]:
np.mean(np.round(block_0306_tassel_count, 0) == np.round(block_0306_tassel_count_alt, 0))

In [ ]:
# save these true counts in a df for future use

In [ ]:
%%time
# Get the stacks?
block_0306_density_list = []
for conv in block_0306_convs:
    density_values = create_densities_for_subwindows(conv, 24, 300)
    block_0306_density_list.append(density_values)

In [ ]:
len(block_0306_density_list)

In [ ]:
block_0306_density_list[0].shape

In [ ]:
# need to stack this list
block_0306_all_density_stack = np.stack(block_0306_density_list, axis = 1)

In [ ]:
block_0306_all_density_stack.shape

In [ ]:
np.array(block_0306_tassel_count_alt)

In [ ]:
np.sum(block_0306_all_density_stack, axis = 0).reshape(1,-1)

In [ ]:
# Need to figure out a better sanity check

In [ ]:
block_0306_convs[0].shape

In [ ]:
all_subwindowed_densitities_im_1 = create_densities_for_subwindows(block_0306_convs[0], stride = 24, kernel_size = 300)

In [ ]:
np.array(all_subwindowed_densitities_im_1).shape

In [ ]:
np.array(all_subwindowed_densitities_im_1).reshape(-1,1).shape

In [ ]:
time_period_name = ['t_' + str(i) for i in range(1, len(block_0306_tassel_count_alt)+1)]
# time_period_name

In [ ]:
true_densities_0306_df = pd.DataFrame(zip(time_period_name, block_0306_tassel_count_alt), columns = ['Time', "True_density"]) 

In [ ]:
true_densities_0306_df.head()

In [ ]:
true_densities_0306_df.to_csv("data/test_block_true_counts/block_0306_true_counts.csv", index = False)

In [ ]:
# What other sanity checks can we perform to ensure the subwindowing is right?

In [ ]:
# We have verified all written codes previously, so let's only verify that the stacking is indeed right

In [ ]:
block_0306_all_density_stack.shape

In [ ]:
len(block_0306_density_list)

In [ ]:
%%time
for i in range(len(block_0306_density_list)):
    print(np.mean(block_0306_density_list[i] == block_0306_all_density_stack[:, i, :]))   

In [ ]:
# Stacking is corretly dine

In [ ]:
# save the final stack as an np array?

In [ ]:
block_0306_all_density_stack.shape

In [ ]:
np.save("data/densities_overlapping/all_densities_block_0306.npy", block_0306_all_density_stack)

In [ ]:
# sanity check
loaded_density_blk_0306 = np.load("data/densities_overlapping/all_densities_block_0306.npy")

In [ ]:
np.mean(loaded_density_blk_0306 == block_0306_all_density_stack)

In [ ]:
# Let's paste from the original notebook, it'll be easier